Models with User Feedback Value

This activity takes a similar approach to using linear regression in filling in missing ratings.  Here, you assume the users have been asked to provide different `slick` and `lofi` scores when signing up for your streaming service.  The goal is to use these ratings across users to build regression models with `slick` and `lofi` as input and each artist as a target.

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [4]:
data = {
    'Michael Jackson': [3.0, np.nan, 2.0, 3.0, 1.0],
    'Clint Black': [4.0, 9.0, 5.0, np.nan, 1.0],
    'Dropdead': [np.nan, np.nan, 8.0, 9.0, np.nan],
    'Anti-Cimex': [4.0, 3.0, 9.0, 4.0, 9.0],
    'Cardi B': [4.0, 8.0, np.nan, 9.0, 5.0],
    'slick': [5, 7, 3, 5, 1],
    'lofi': [5, 4, 6, 5, 8]
}

reviews = pd.DataFrame(data, index=['Alfred', 'Mandy', 'Lenny', 'Joan', 'Tino'])
reviews

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B,slick,lofi
Alfred,3.0,4.0,NaN,4.0,4.0,5,5
Mandy,NaN,9.0,NaN,3.0,8.0,7,4
Lenny,2.0,5.0,8.0,9.0,NaN,3,6
Joan,3.0,NaN,9.0,4.0,9.0,5,5
Tino,1.0,1.0,NaN,9.0,5.0,1,8


In [6]:
# Michael Jackson Model
X = reviews.dropna(subset = ['Michael Jackson'])[['slick', 'lofi']]
y = reviews['Michael Jackson'].dropna()
mj_lr = LinearRegression().fit(X, y)
mandy_predict = mj_lr.predict(reviews[reviews['Michael Jackson'].isnull()][['slick', 'lofi']])

reviews.loc['Mandy', 'Michael Jackson'] = mandy_predict

reviews

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B,slick,lofi
Alfred,3.0,4.0,NaN,4.0,4.0,5,5
Mandy,4.0,9.0,NaN,3.0,8.0,7,4
Lenny,2.0,5.0,8.0,9.0,NaN,3,6
Joan,3.0,NaN,9.0,4.0,9.0,5,5
Tino,1.0,1.0,NaN,9.0,5.0,1,8


In [8]:
coefs = []
for col in reviews.columns:
    try:
        X = reviews.dropna(subset = [col])[['slick', 'lofi']]
        y = reviews[col].dropna()
        mj_lr = LinearRegression().fit(X, y)
        newx = reviews[reviews[col].isnull()][['slick', 'lofi']]
        reviews.loc[reviews[col].isnull(), col] = mj_lr.predict(newx)
        coefs.append(list(mj_lr.coef_))
    except:
        pass

reviews

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B,slick,lofi
Alfred,3.0,4.0,9.0,4.0,4.0,5,5
Mandy,4.0,9.0,10.0,3.0,8.0,7,4
Lenny,2.0,5.0,8.0,9.0,5.0,3,6
Joan,3.0,6.0,9.0,4.0,9.0,5,5
Tino,1.0,1.0,6.8,9.0,5.0,1,8
